<a href="https://colab.research.google.com/github/szyrek/sentiment_anal/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install tensorflow
!pip install nltk
!pip install pandas
!pip install numpy
!pip install tensorflow-datasets

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import json
from google.colab import drive

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Make sure we use correct TensorFlow version and we're running on GPU
print("TensorFlow version:", tf.__version__)
gpus = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", gpus)

# This will print the name and type of all available devices
print("Device List:", tf.config.experimental.list_physical_devices())

# Set batch size accordingly
if gpus > 0:
    reasonable_batch=512
else:
    reasonable_batch=64

print("Setting training batch_size to:", reasonable_batch)

TensorFlow version: 2.12.0
Num GPUs Available:  0
Device List: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Setting training batch_size to: 64


In [ ]:
# Load IMDb dataset
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

# Prepare data for training
# TensorFlow Datasets provides the IMDb reviews in a tensor format. We need to convert these into numpy arrays and then into a format suitable for our model
train_data = tfds.as_numpy(train_dataset)
test_data = tfds.as_numpy(test_dataset)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteERR5VF/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteERR5VF/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteERR5VF/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# Initialize lists to hold processed texts and labels
train_texts, train_labels = [], []
test_texts, test_labels = [], []

# Extract texts and labels from the training and test datasets correctly
for text, label in train_data:
    train_texts.append(text.decode('utf-8'))  # Decode bytes to string
    train_labels.append(label)

for text, label in test_data:
    test_texts.append(text.decode('utf-8'))  # Decode bytes to string
    test_labels.append(label)

# Convert labels into numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# Text preprocessing
stop_words = set(stopwords.words('english'))
processed_train_texts = []
processed_test_texts = []

for text in train_texts:
    words = text.split()
    words_filtered = [word for word in words if word.lower() not in stop_words]
    processed_train_texts.append(" ".join(words_filtered))

for text in test_texts:
    words = text.split()
    words_filtered = [word for word in words if word.lower() not in stop_words]
    processed_test_texts.append(" ".join(words_filtered))

In [ ]:
# Tokenization and padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(processed_train_texts)
train_sequences = tokenizer.texts_to_sequences(processed_train_texts)
test_sequences = tokenizer.texts_to_sequences(processed_test_texts)

train_padded = pad_sequences(train_sequences, maxlen=200)
test_padded = pad_sequences(test_sequences, maxlen=200)

In [ ]:
# Building the model
model = Sequential([
    Embedding(5000, 128, input_length=200),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_padded, train_labels, batch_size=reasonable_batch, epochs=10, validation_data=(test_padded, test_labels))

Epoch 1/10
49/49 [==============================] - 39s 782ms/step - loss: 0.2330 - accuracy: 0.9112 - val_loss: 0.2965 - val_accuracy: 0.8783
Epoch 2/10
49/49 [==============================] - 40s 827ms/step - loss: 0.2061 - accuracy: 0.9234 - val_loss: 0.3088 - val_accuracy: 0.8759
Epoch 3/10
49/49 [==============================] - 36s 722ms/step - loss: 0.1862 - accuracy: 0.9318 - val_loss: 0.3080 - val_accuracy: 0.8773
Epoch 4/10
49/49 [==============================] - 37s 766ms/step - loss: 0.1661 - accuracy: 0.9399 - val_loss: 0.3185 - val_accuracy: 0.8767
Epoch 5/10
49/49 [==============================] - 36s 736ms/step - loss: 0.1487 - accuracy: 0.9471 - val_loss: 0.3296 - val_accuracy: 0.8758
Epoch 6/10
49/49 [==============================] - 37s 755ms/step - loss: 0.1317 - accuracy: 0.9544 - val_loss: 0.3982 - val_accuracy: 0.8694
Epoch 7/10
49/49 [==============================] - 34s 699ms/step - loss: 0.1182 - accuracy: 0.9597 - val_loss: 0.3723 - val_accuracy: 0.8723

In [ ]:
# Evaluate the model
scores = model.evaluate(test_padded, test_labels)
print(f"Test Accuracy: {scores[1]*100}%")

782/782 [==============================] - 51s 65ms/step - loss: 0.5660 - accuracy: 0.8511
Test Accuracy: 85.10800004005432%


In [ ]:
# Mount Google Drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Save the model
model_save_path = "/content/drive/My Drive/my_model.h5"
model.save(model_save_path)

# Save tokenizer
tokenizer_json = tokenizer.to_json()
with open("/content/drive/My Drive/tokenizer.json", 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import tokenizer_from_json

# Load the model
model = load_model("/content/drive/My Drive/my_model.h5")

# Load the tokenizer
with open("/content/drive/My Drive/tokenizer.json") as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

In [ ]:
# Evaluate single text
text = "I wanted to pee almost since the beginning, but it was too captivating to miss any minute of it. Great flick, loved it!"

def preprocess_text(text):
    # Assuming 'tokenizer' is your tokenizer and is already fitted to your training data
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    tokenized_text = tokenizer.texts_to_sequences([" ".join(filtered_words)])
    padded_text = pad_sequences(tokenized_text, maxlen=200)  # Use the same maxlen as during training
    return padded_text

# Preprocess the text
processed_text = preprocess_text(text)

# Predict (use the loaded model if you're in a new session)
prediction = model.predict(processed_text)

print("Prediction (closer to 1 means positive sentiment):", prediction[0][0])

1/1 [==============================] - 0s 383ms/step
Prediction (closer to 1 means positive sentiment): 0.99185354
